In [1]:
using Distributed
const np = 3
addprocs(np)
@everywhere const np = nworkers()

In [2]:
@everywhere using DistributedArrays
@everywhere using DistributedArrays.SPMD
@everywhere using FFTW
@everywhere using LinearAlgebra
@everywhere using ParallelDataTransfer

@everywhere const bnmesh = 150
@everywhere const bcnmesh = convert(Int64,bnmesh/2) + 1

@everywhere const nkmax = 21#42
@everywhere const skip = 1
@everywhere const ndkmax = skip*nkmax

@everywhere pids = workers()
@everywhere pstart = minimum(pids)

bk = Array{Float64,3}(undef,nkmax,nkmax,nkmax)
@defineat pstart bk = Array{Float64,3}(undef,nkmax,nkmax,nkmax)
tk = Array{Float64,4}(undef,nkmax,nkmax,nkmax,nkmax)
@defineat pstart tk = Array{Float64,4}(undef,nkmax,nkmax,nkmax,nkmax)
qk = Array{Float64,5}(undef,nkmax,nkmax,nkmax,nkmax,nkmax)
@defineat pstart qk = Array{Float64,5}(undef,nkmax,nkmax,nkmax,nkmax,nkmax)
pk = Array{Float64,6}(undef,nkmax,nkmax,nkmax,nkmax,nkmax,nkmax)
@defineat pstart pk = Array{Float64,6}(undef,nkmax,nkmax,nkmax,nkmax,nkmax,nkmax)

delkix = dzeros(Complex{Float64}, (bcnmesh, bnmesh, bnmesh, nkmax), pids, [np,1,1,1])
dk = dzeros(Complex{Float64}, (bcnmesh, bnmesh, bnmesh), pids, [np,1,1])
delkir = dzeros((bnmesh,bnmesh,bnmesh,nkmax), pids, [np,1,1,1])
dr = dzeros((bnmesh,bnmesh,bnmesh), pids, [np,1,1]);

@defineat pstart bkfname = "NkOut/Nmodes_bk_nkmax=$(nkmax)_s=$(skip).dat"
@defineat pstart tkfname = "NkOut/Nmodes_tk_nkmax=$(nkmax)_s=$(skip).dat"
@defineat pstart qkfname = "NkOut/Nmodes_4k_nkmax=$(nkmax)_s=$(skip).dat"
@defineat pstart pkfname = "NkOut/Nmodes_5k_nkmax=$(nkmax)_s=$(skip).dat"
@defineat pstart bkf = open(bkfname, "w")
@defineat pstart tkf = open(tkfname, "w")
@defineat pstart qkf = open(qkfname, "w")
@defineat pstart pkf = open(pkfname, "w")

Future(2, 1, 128, nothing)

In [3]:
@everywhere function fourier_space_density!(delkix::DArray)
    for k = 1:(bnmesh)
        kz = k - 1
        if (k-1) >= bcnmesh
            kz = kz - bnmesh
        end
        for j = 1:(bnmesh)
            ky = j - 1
            if (j-1) >= bcnmesh
                ky = ky - bnmesh
            end
            for i = 1:length(localindices(delkix)[1])
                kx = i - 1 + localindices(delkix)[1][1] - 1
                magnitude = sqrt(kx^2+ky^2+kz^2)
                numnk = convert(Int64,round(magnitude/skip))

                if numnk <= nkmax && numnk > 0
                    delkix[:L][i,j,k,numnk] = 1.0 + 0.0im
                end
            end
        end
    end
end
spmd(fourier_space_density!, delkix; pids=pids)
@spawnat pstart fill!(delkix[:L][1,1,1,:],0.0 + 0.0im);

In [4]:
@everywhere function transpose3D(x)
    permutedims(x,(2,1,3))
end


@everywhere function alltoalltranspose3D(x, nx, recievearray; pids=workers())
    pstart = minimum(pids)
    np = length(pids)
    rank = myid() - pstart + 1
    ny = size(x,2)
    extrax = nx % np
    extray = ny % np
    if rank <= extrax
        haveextra = true
    else
        haveextra = false
    end
    if rank <= extray
        haveextray = true
    else
        haveextray = false
    end
    
    if haveextra
        dx = size(x,1) - 1
    else
        dx = size(x,1)
    end
    nz = size(x,3)
    dy = convert(Int64,floor(ny/np))
    

    xt = transpose3D(x)

    i=1
    while i < np
        i *= 2
    end
    if i == np
        pow2 = true
    else
        pow2 = false
    end
    
    if haveextra
        if haveextray
            offset = rank - 1
            recievearray[:,(1+dx*(rank-1)+offset):dx*rank+(offset+1),:] .= 
                        xt[(1+dy*(rank-1)+offset):dy*rank+(offset+1),:,:] 
        else
            offsetx = rank - 1
            offsety = extray
            recievearray[:,(1+dx*(rank-1)+offsetx):dx*rank+(offsetx+1),:] .= 
                        xt[(1+dy*(rank-1)+offsety):dy*rank+(offsety),:,:] 
        end
    else
        if haveextray
            offsetx = extrax
            offsety = rank - 1
            recievearray[:,(1+dx*(rank-1)+offsetx):dx*rank+(offsetx),:] .= 
                        xt[(1+dy*(rank-1)+offsety):dy*rank+(offsety+1),:,:] 
        else
            offsetx = extrax
            offsety = extray
            recievearray[:,(1+dx*(rank-1)+offsetx):dx*rank+offsetx,:] .= 
                        xt[(1+dy*(rank-1)+offsety):dy*rank+offsety,:,:] 
        end
    end
    
    for i = 1:(np-1)
        if pow2
            source = destination = ((rank-1) ⊻ i) + 1
        else
            source = (rank - i - 1 + np ) % np + 1
            destination = (rank + i - 1 + np) % np + 1
        end
        if destination <= extray
            offset = destination - 1
            SPMD.sendto(destination + pstart - 1, xt[(1+dy*(destination-1)+offset):dy*destination+(offset+1),:,:])
        else
            offset = extray
            SPMD.sendto(destination + pstart - 1, xt[(1+dy*(destination-1)+offset):dy*destination+offset,:,:])
        end
        
        if source <= extrax
            offset = source - 1
            recievearray[:,(1+dx*(source-1)+offset):dx*source+(offset+1),:] .= SPMD.recvfrom(source + pstart - 1)
        else
            offset = extrax
            recievearray[:,(1+dx*(source-1)+offset):dx*source+offset,:] .= SPMD.recvfrom(source + pstart - 1)
        end
        barrier(; pids=pids)
    end
end


@everywhere function delta_ki_fft!(dk,delkix,dr,delkir)
    pids = procs(dk)
    pstart = minimum(pids)
    
    haveextra = ((myid() - pstart + 1) <= rem(size(delkix,2), np))
    if haveextra
        transpose_size = (div(bnmesh, np) + 1, bcnmesh, bnmesh)
    else
        transpose_size = (div(bnmesh, np), bcnmesh, bnmesh)
    end
    
    z = zeros(Complex{Float64}, transpose_size)
    
    function generate_plans(delkix)
        
        plan1 = plan_bfft!(delkix[:L], (2,3), flags=FFTW.MEASURE)
        plan2 = plan_brfft(z, 2*bcnmesh - 2, (2,), flags=FFTW.MEASURE)
        return [plan1, plan2]
    end

    function apply_plans(dk,dr,plans)
        
        plans[1]*dk[:L]
        alltoalltranspose3D(dk[:L], size(dk,1), z)
        w = plans[2]*z
        alltoalltranspose3D(w, size(dk,2), dr[:L])
    end

    plans = generate_plans(dk)
    
    for nkindx = 1:nkmax
        if myid() == pstart
            println("FFT for nkindx=$nkindx")
        end        
        @inbounds dk[:L] = delkix[:L][:,:,:,nkindx]
        apply_plans(dk,dr,plans)
        @inbounds delkir[:L][:,:,:,nkindx] = dr[:L]
    end
end

spmd(delta_ki_fft!, dk, delkix, dr, delkir; pids=pids)

      From worker 2:	FFT for nkindx=1
      From worker 2:	FFT for nkindx=2
      From worker 2:	FFT for nkindx=3
      From worker 2:	FFT for nkindx=4
      From worker 2:	FFT for nkindx=5
      From worker 2:	FFT for nkindx=6
      From worker 2:	FFT for nkindx=7
      From worker 2:	FFT for nkindx=8
      From worker 2:	FFT for nkindx=9
      From worker 2:	FFT for nkindx=10
      From worker 2:	FFT for nkindx=11
      From worker 2:	FFT for nkindx=12
      From worker 2:	FFT for nkindx=13
      From worker 2:	FFT for nkindx=14
      From worker 2:	FFT for nkindx=15
      From worker 2:	FFT for nkindx=16
      From worker 2:	FFT for nkindx=17
      From worker 2:	FFT for nkindx=18
      From worker 2:	FFT for nkindx=19
      From worker 2:	FFT for nkindx=20
      From worker 2:	FFT for nkindx=21


In [6]:
@everywhere function compute_pk_bk!(bk,delkir,dr)
    pids = procs(dr)[:,1]
    #println(pids)
    pstart = minimum(pids)
    function pk_bk(j1,j2,j3)
        kx = skip*j1   
        ky = j2*skip        
        kz = j3*skip
        if j3 > 0
            @inbounds dr[:L] = delkir[:L][:,:,:,j1] .*
                               delkir[:L][:,:,:,j2] .*
                               delkir[:L][:,:,:,j3]
            barrier(;pids=pids)
            if myid() == pstart
                bk[j1,j2,j3] = sum(dr)/bnmesh^3
                write(bkf, "$kx $ky $kz $(bk[j1,j2,j3]) \n")
            end
            barrier(;pids=pids)
            
            if myid() == pstart
                if j1 == j2 && j2 == j3
                    flush(bkf)
                    println("Bk for nkindx=$j1")
                end
            end
        end
    end

    for j1 = 1:nkmax, j2 = 1:j1, j3 = (j1-j2):j2
        pk_bk(j1,j2,j3)
    end
end

spmd(compute_pk_bk!, bk, delkir, dr; pids=pids)
fetch(@spawnat pstart close(bkf))

In [8]:
@everywhere function compute_tk!(tk,delkir,dr)
    pids = procs(dr)[:,1]
    pstart = minimum(pids)
    function _tk(j1,j2,j3,j4)
        kx = skip*j1   
        ky = j2*skip        
        kz = j3*skip
        kq = j4*skip
        if j4 > 0
            @inbounds dr[:L] = delkir[:L][:,:,:,j1] .*
                               delkir[:L][:,:,:,j2] .*
                               delkir[:L][:,:,:,j3] .*
                               delkir[:L][:,:,:,j4]
            barrier(;pids=pids)
            if myid() == pstart
                tk[j1,j2,j3,j4] = sum(dr)/bnmesh^3
                write(tkf, "$kx $ky $kz $kq $(tk[j1,j2,j3,j4]) \n")
            end
            barrier(;pids=pids)
            
            if myid() == pstart
                if j1 == j2 && j2 == j3 && j3 == j4
                    flush(tkf)
                    println("Tk for nkindx=$j1")
                end
            end
        end
    end

    for j1 = 1:nkmax, j2 = 1:j1, j3 = 1:j2, j4 = (j1-j2-j3):j3
        _tk(j1,j2,j3,j4)
    end
end

spmd(compute_tk!, tk, delkir, dr; pids=pids)
fetch(@spawnat pstart close(tkf))

In [10]:
@everywhere function compute_4k!(qk,delkir,dr)
    pids = procs(dr)[:,1]
    pstart = minimum(pids)
    function _qk(j1,j2,j3,j4,j5)
        kx = skip*j1   
        ky = j2*skip        
        kz = j3*skip
        kq = j4*skip
        k5 = j5*skip
        if j5 > 0
            @inbounds dr[:L] = delkir[:L][:,:,:,j1] .*
                               delkir[:L][:,:,:,j2] .*
                               delkir[:L][:,:,:,j3] .*
                               delkir[:L][:,:,:,j4] .*
                               delkir[:L][:,:,:,j5]
            barrier(;pids=pids)
            if myid() == pstart
                qk[j1,j2,j3,j4,j5] = sum(dr)/bnmesh^3
                write(qkf, "$kx $ky $kz $kq $k5 $(qk[j1,j2,j3,j4,j5]) \n")
            end
            barrier(;pids=pids)
            
            if myid() == pstart
                if j1 == j2 && j2 == j3 && j3 == j4 && j4 == j5
                    flush(qkf)
                    println("Qk for nkindx=$j1")
                end
            end
        end
    end

    for j1 = 1:nkmax, j2 = 1:j1, j3 = 1:j2, j4 = 1:j3, j5 = (j1-j2-j3-j4):j4
        _qk(j1,j2,j3,j4,j5)
    end
end

spmd(compute_4k!, qk, delkir, dr; pids=pids)
fetch(@spawnat pstart close(qkf))

      From worker 2:	Qk for nkindx=1
      From worker 2:	Qk for nkindx=2
      From worker 2:	Qk for nkindx=3
      From worker 2:	Qk for nkindx=4
      From worker 2:	Qk for nkindx=5
      From worker 2:	Qk for nkindx=6
      From worker 2:	Qk for nkindx=7
      From worker 2:	Qk for nkindx=8
      From worker 2:	Qk for nkindx=9
      From worker 2:	Qk for nkindx=10
      From worker 2:	Qk for nkindx=11
      From worker 2:	Qk for nkindx=12
      From worker 2:	Qk for nkindx=13
      From worker 2:	Qk for nkindx=14
      From worker 2:	Qk for nkindx=15
      From worker 2:	Qk for nkindx=16
      From worker 2:	Qk for nkindx=17
      From worker 2:	Qk for nkindx=18
      From worker 2:	Qk for nkindx=19
      From worker 2:	Qk for nkindx=20
      From worker 2:	Qk for nkindx=21


In [12]:
@everywhere function compute_5k!(pk,delkir,dr)
    pids = procs(dr)[:,1]
    pstart = minimum(pids)
    function _pk(j1,j2,j3,j4,j5,j6)
        kx = skip*j1   
        ky = j2*skip        
        kz = j3*skip
        kq = j4*skip
        k5 = j5*skip
        k6 = j6*skip
        if j6 > 0
            @inbounds dr[:L] = delkir[:L][:,:,:,j1] .*
                               delkir[:L][:,:,:,j2] .*
                               delkir[:L][:,:,:,j3] .*
                               delkir[:L][:,:,:,j4] .*
                               delkir[:L][:,:,:,j5] .*
                               delkir[:L][:,:,:,j6]
            barrier(;pids=pids)
            if myid() == pstart
                pk[j1,j2,j3,j4,j5,j6] = sum(dr)/bnmesh^3
                write(pkf, "$kx $ky $kz $kq $k5 $k6 $(pk[j1,j2,j3,j4,j5,j6]) \n")
            end
            barrier(;pids=pids)
            
            if myid() == pstart
                if j1 == j2 && j2 == j3 && j3 == j4 && j4 == j5 && j5 == j6
                    flush(pkf)
                    println("5k for nkindx=$j1")
                end
            end
        end
    end

    for j1 = 1:nkmax, j2 = 1:j1, j3 = 1:j2, j4 = 1:j3, j5 = 1:j4, j6 = (j1-j2-j3-j4-j5):j5
        _pk(j1,j2,j3,j4,j5,j6)
    end
end

spmd(compute_5k!, pk, delkir, dr; pids=pids)
fetch(@spawnat pstart close(pkf))

      From worker 2:	5k for nkindx=1
      From worker 2:	5k for nkindx=2
      From worker 2:	5k for nkindx=3
      From worker 2:	5k for nkindx=4
      From worker 2:	5k for nkindx=5
      From worker 2:	5k for nkindx=6
      From worker 2:	5k for nkindx=7
      From worker 2:	5k for nkindx=8
      From worker 2:	5k for nkindx=9
      From worker 2:	5k for nkindx=10
      From worker 2:	5k for nkindx=11
      From worker 2:	5k for nkindx=12
      From worker 2:	5k for nkindx=13
      From worker 2:	5k for nkindx=14
      From worker 2:	5k for nkindx=15
      From worker 2:	5k for nkindx=16
      From worker 2:	5k for nkindx=17
      From worker 2:	5k for nkindx=18
      From worker 2:	5k for nkindx=19
      From worker 2:	5k for nkindx=20
      From worker 2:	5k for nkindx=21
